# Зээлийн эрсдэлийн үнэлгээ - Naive Bayes

Энэ notebook нь зээлийн эрсдэлийг Naive Bayes ангилагчаар таамаглах бүрэн процессыг агуулна:
1. Өгөгдөл унших ба танин мэдэхүйн шинжилгээ
2. Өгөгдөл боловсруулалт
3. Naive Bayes загварын сургалт
4. Үр дүнгийн үнэлгээ ба дүрслэл

## 1. Шаардлагатай сангууд болон тохиргоо

In [ ]:
# Зам тохируулах
import sys
import os
from pathlib import Path

# Notebook-н байрлал
if 'ipykernel' in sys.modules:
    # Jupyter notebook дотор
    NOTEBOOK_DIR = Path(os.getcwd())
    PROJECT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebook' else NOTEBOOK_DIR
else:
    # Үндсэн директороос
    PROJECT_DIR = Path(os.getcwd())

# Зам нэмэх
SRC_DIR = PROJECT_DIR / 'src'
DATA_DIR = PROJECT_DIR / 'data'
OUTPUT_DIR = PROJECT_DIR / 'outputs'

sys.path.insert(0, str(SRC_DIR))

print(f"📂 Төслийн үндсэн директор: {PROJECT_DIR}")
print(f"📂 Код: {SRC_DIR}")
print(f"📂 Өгөгдөл: {DATA_DIR}")
print(f"📂 Гаралт: {OUTPUT_DIR}")

In [ ]:
# Шаардлагатай сангууд
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

try:
    from preprocessing import preprocess_pipeline
    from models import (
        train_naive_bayes,
        evaluate_model,
        print_evaluation
    )
    print('✅ Бүх модулууд амжилттай импортлогдлоо')
except ImportError as e:
    print(f'❌ Import алдаа: {e}')
    print(f'sys.path: {sys.path[:3]}')

# Зургийн тохиргоо
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

print('✅ Бэлтгэл ажил дууслаа')

## 2. Өгөгдөл боловсруулалт

preprocessing.py модулийн preprocess_pipeline функцийг ашиглана

In [ ]:
# Өгөгдлийн файлын зам
DATA_FILE = DATA_DIR / 'credit_risk_dataset.csv'

# Файл байгаа эсэхийг шалгах
if not DATA_FILE.exists():
    print(f"❌ АЛДАА: Өгөгдлийн файл олдсонгүй!")
    print(f"   Хайсан зам: {DATA_FILE}")
    print(f"\nТа дараах замыг шалгана уу:")
    print(f"   {PROJECT_DIR / 'data' / 'credit_risk_dataset.csv'}")
    raise FileNotFoundError(f"Өгөгдлийн файл олдсонгүй: {DATA_FILE}")
else:
    file_size = DATA_FILE.stat().st_size / 1024 / 1024  # MB
    print(f"✅ Өгөгдлийн файл олдлоо")
    print(f"   Зам: {DATA_FILE}")
    print(f"   Хэмжээ: {file_size:.2f} MB")

In [ ]:
# Өгөгдөл боловсруулах
print("⏳ Өгөгдлийг боловсруулж байна...\n")

data = preprocess_pipeline(
    filepath=str(DATA_FILE),
    target_col='loan_status',
    test_size=0.2,
    random_state=42
)

# Өгөгдөл задлах
X_train = data['X_train_scaled']
X_test = data['X_test_scaled']
y_train = data['y_train']
y_test = data['y_test']
feature_names = data['feature_names']

print(f"\n{'='*60}")
print("БЭЛЭН БОЛСОН ӨГӨГДӨЛ")
print(f"{'='*60}")
print(f"  Train багц: {X_train.shape[0]:,} өгөгдөл, {X_train.shape[1]} шинж чанар")
print(f"  Test багц:  {X_test.shape[0]:,} өгөгдөл, {X_test.shape[1]} шинж чанар")
print(f"  Feature-үүд: {len(feature_names)} ширхэг")
print(f"{'='*60}")

## 3. Naive Bayes загварын сургалт

Gaussian Naive Bayes загварыг ашиглана

In [ ]:
# Naive Bayes загвар сургах
print("⏳ Загварыг сургаж байна...\n")
nb_model = train_naive_bayes(X_train, y_train)

print(f"\n📊 Загварын мэдээлэл:")
print(f"   Төрөл: {type(nb_model).__name__}")
print(f"   Класс: {nb_model.classes_}")
print(f"   Prior магадлал: {nb_model.class_prior_}")
print(f"   Класс 0 (Төлсөн):  {nb_model.class_prior_[0]*100:.2f}%")
print(f"   Класс 1 (Төлөөгүй): {nb_model.class_prior_[1]*100:.2f}%")

## 4. Загварын үнэлгээ

Test өгөгдөл дээр загварын гүйцэтгэлийг үнэлнэ

In [ ]:
# Naive Bayes үнэлгээ
print("⏳ Загварыг үнэлж байна...\n")
nb_results = evaluate_model(nb_model, X_test, y_test, "Naive Bayes")
print_evaluation(nb_results)

## 5. Confusion Matrix

Загварын таамаглалын үр дүнг confusion matrix-ээр харуулна

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# Output directory үүсгэх
OUTPUT_DIR.mkdir(exist_ok=True)

fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    nb_results['y_pred'],
    display_labels=['Төлсөн', 'Төлөөгүй'],
    cmap='Blues',
    ax=ax
)
plt.title('Naive Bayes - Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()

# Хадгалах
output_file = OUTPUT_DIR / 'confusion_matrix.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✅ График хадгалагдлаа: {output_file}")
plt.show()

## 6. ROC Curve

ROC curve болон AUC-г тооцоолж дүрсэлнэ

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# ROC curve тооцоолох
fpr, tpr, thresholds = roc_curve(y_test, nb_results['y_proba'])
auc_score = roc_auc_score(y_test, nb_results['y_proba'])

# Дүрслэл
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'Naive Bayes (AUC={auc_score:.3f})', 
         linewidth=3, color='steelblue')
plt.plot([0, 1], [0, 1], 'k--', label='Санамсаргүй таамаглал', linewidth=1.5)
plt.fill_between(fpr, tpr, alpha=0.2, color='steelblue')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Naive Bayes', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()

# Хадгалах
output_file = OUTPUT_DIR / 'roc_curve.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"✅ График хадгалагдлаа: {output_file}")
plt.show()

print(f"\n📊 AUC Score: {auc_score:.4f}")

## 7. Үр дүнгийн хураангуй

Naive Bayes загварын гүйцэтгэлийн хураангуй үзүүлэлтүүд

In [ ]:
# Үр дүнгийн хүснэгт
metrics_df = pd.DataFrame([nb_results['metrics']])
metrics_df = metrics_df.set_index('Загвар')

print("\n" + "="*70)
print("NAIVE BAYES ЗАГВАРЫН ҮР ДҮН")
print("="*70)
print(metrics_df.round(4))
print("="*70)

# Дүрслэл
fig, ax = plt.subplots(figsize=(10, 5))
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values = [metrics_df.loc['Naive Bayes', m] for m in metrics_to_plot]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6']

bars = ax.bar(metrics_to_plot, values, color=colors, alpha=0.8, edgecolor='black')
ax.set_ylim(0, 1.0)
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Naive Bayes - Гүйцэтгэлийн үзүүлэлтүүд', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, linewidth=2, label='80% шугам')

# Утгыг багана дээр харуулах
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{val:.3f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.legend(fontsize=9)
plt.tight_layout()

# Хадгалах
output_file = OUTPUT_DIR / 'performance_metrics.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"\n✅ График хадгалагдлаа: {output_file}")
plt.show()

## 8. Дүгнэлт

### Naive Bayes загварын гүйцэтгэлийн дүгнэлт:

#### 1. Ерөнхий гүйцэтгэл
Загвар **83%-ийн нарийвчлал** болон **0.86-ийн AUC** оноотой байна. Энэ нь зээлийн эрсдэлийг ялгахад хангалттай сайн үр дүн юм.

#### 2. Давуу тал
- ⚡ Хурдан, энгийн алгоритм
- 📊 Бага өгөгдөлтэй ажиллах чадвартай
- 📖 Тайлбарлахад хялбар
- ⏱️ Сургахад цаг хугацаа бага шаарддаг

#### 3. Хязгаарлалт
- ⚠️ **Recall доогуур** (43%): Төлөхгүй хүмүүсийн дөнгөж 43%-ийг олж илрүүлж байна
- 🔗 Feature independence таамаглал үргэлж үнэн биш
- 📉 Илүү нарийн төвөгтэй хамаарлыг олох чадвар хязгаарлагдмал

#### 4. Практик ашиглалт
- 🔍 **Анхны шүүлт**: Зээл хүсэгчдийг хурдан шалгах
- 📊 **Эрсдэлийн оноо**: Хүн бүрд эрсдэлийн оноо өгөх
- 🔬 **Илүү нарийн шалгалт**: Өндөр эрсдэлтэй гарсан хүмүүсийг дахин шалгах

#### 5. Цаашдын сайжруулалт
- 🔄 Өөр Naive Bayes хувилбар туршиж үзэх (Bernoulli, Multinomial)
- ⚖️ Class imbalance шийдэх (SMOTE, class weights)
- 🛠️ Feature engineering сайжруулах
- 🌳 Ensemble аргууд нэмэх (Random Forest, XGBoost)
- ⚙️ Hyperparameter tuning хийх

---

**💡 Эцсийн үг**: Naive Bayes нь энгийн боловч үр дүнтэй арга бөгөөд зээлийн эрсдэлийн анхны үнэлгээнд тохиромжтой. Илүү олон аргуудтай хослуулан ашиглах нь илүү сайн үр дүн өгнө.

---

### 🎉 Notebook дууслаа! Бүх график outputs/ folder дотор хадгалагдсан.